### Wikipedia Scraping

In [151]:
!pip install html5lib
!pip install lxml
!pip install beautifulSoup4

In [2]:
import html5lib
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [1]:
url = 'https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
tables = pd.read_html(url) 
tables[0]

,Film,Year,Awards,Nominations
0,CODA,2021,3,3
1,Dune,2021,6,10
2,The Eyes of Tammy Faye,2021,2,2
3,No Time to Die,2021,1,3
4,The Windshield Wiper,2021,1,1
...,...,...,...,...
1342,The Yankee Doodle Mouse,1943,1,1
1343,The Yearling,1946,2,7
1344,"Yesterday, Today and Tomorrow (Ieri, oggi, dom...",1964,1,1
1345,You Can't Take It with You,1938,2,7


In [4]:
oscars_df = tables[0]

In [5]:
oscars_df

,Film,Year,Awards,Nominations
0,CODA,2021,3,3
1,Dune,2021,6,10
2,The Eyes of Tammy Faye,2021,2,2
3,No Time to Die,2021,1,3
4,The Windshield Wiper,2021,1,1
...,...,...,...,...
1342,The Yankee Doodle Mouse,1943,1,1
1343,The Yearling,1946,2,7
1344,"Yesterday, Today and Tomorrow (Ieri, oggi, dom...",1964,1,1
1345,You Can't Take It with You,1938,2,7


In [6]:
# exporting raw dataframe
oscars_df.to_csv('oscars_raw_data.csv', index=False)

### BoxMojo Scraping

In [69]:
def scrape():
    """
    Runs through a range of years to pull the box office revenue data for each given year
    """
    years = [str(a) for a in range(1977,2023)]
    df_list = []
    for year in years:
        url = 'https://www.boxofficemojo.com/year/' + year + '/?grossesOption=totalGrosses'
        print('Box Office data for %s scraped' % year)
        tables = pd.read_html(url)
        df = tables[0]
        df_list.append(df)
    main_df = pd.concat(df_list)
    main_df.to_csv('boxoffice_data_raw.csv')

In [70]:
main_df = []
scrape()

Box Office data for 1977 scraped
Box Office data for 1978 scraped
Box Office data for 1979 scraped
Box Office data for 1980 scraped
Box Office data for 1981 scraped
Box Office data for 1982 scraped
Box Office data for 1983 scraped
Box Office data for 1984 scraped
Box Office data for 1985 scraped
Box Office data for 1986 scraped
Box Office data for 1987 scraped
Box Office data for 1988 scraped
Box Office data for 1989 scraped
Box Office data for 1990 scraped
Box Office data for 1991 scraped
Box Office data for 1992 scraped
Box Office data for 1993 scraped
Box Office data for 1994 scraped
Box Office data for 1995 scraped
Box Office data for 1996 scraped
Box Office data for 1997 scraped
Box Office data for 1998 scraped
Box Office data for 1999 scraped
Box Office data for 2000 scraped
Box Office data for 2001 scraped
Box Office data for 2002 scraped
Box Office data for 2003 scraped
Box Office data for 2004 scraped
Box Office data for 2005 scraped
Box Office data for 2006 scraped
Box Office

In [71]:
url = 'https://www.boxofficemojo.com/title/tt0114709/?ref_=bo_se_r_1'
tables = pd.read_html(url) 

In [83]:
tables[0]

,Release Group,Rollout,Markets,Domestic,International,Worldwide
0,Original Release,"November 22-March 27, 1996","EMEA, Domestic","$191,796,233","$52,855,355","$244,651,588"
1,2009 Re-release,"October 1-July 1, 2010",42 markets,"$30,702,446","$10,555,852","$41,258,298"
2,2020 Re-release,"June 11-November 6, 2020","Domestic, APAC","$727,000","$37,172","$764,172"


In [88]:
df = tables[0]

In [92]:
df.iloc[[0]]

,Release Group,Rollout,Markets,Domestic,International,Worldwide
0,Original Release,"November 22-March 27, 1996","EMEA, Domestic","$191,796,233","$52,855,355","$244,651,588"


In [12]:
url = 'https://www.boxofficemojo.com/title/tt0110217/?ref_=bo_se_r_1'
tables = pd.read_html(url) 

In [13]:
df = tables[0]

In [14]:
df.iloc[[0]]

,Area,Release Date,Opening,Gross
0,Domestic,"Aug 18, 1995","$3,673","$10,622"


In [3]:
# opening tmdb to get imdb movie ids
tmdb_df_raw = pd.read_csv('tmdb_raw_data.csv')

In [4]:
film_id = tmdb_df_raw[['imdb_id']]

In [5]:
film_id.head()

,imdb_id
0,tt0114709
1,tt0113497
2,tt0113228
3,tt0114885
4,tt0113041


In [6]:
film_id.shape[0]

9626

In [39]:
def scrape():
    """
    Runs list of imdb ids and gathers the box office data for each movie
    """
    movies = [str(a) for a in film_id.imdb_id]
    df_list = []
    pages_remaining = film_id.shape[0]
    for movie in movies:
        try:
            url = 'https://www.boxofficemojo.com/title/' + movie + '/?ref_=bo_se_r_1'
            tables = pd.read_html(url)
            error_list = []
            df = tables[0]
            df = df.iloc[[0]]
            df['imdb_id'] = movie
            df_list.append(df)
            pages_remaining = pages_remaining - 1
            print(f'{movie} Scraped // Films left to scrape = {pages_remaining}')
        except (ValueError):
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered ValueError // Films left to scrape = {pages_remaining}')
            error_list.append(movie)
        except (URLError):
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered URLError // Films left to scrape = {pages_remaining}')
            error_list.append(movie)
    
    file = open('errors.txt','w')
    file.writelines(movie)
    file.close()
    main_df = pd.concat(df_list)
    main_df.to_csv('revenue_data_raw.csv')

In [40]:
pd.options.mode.chained_assignment = None

In [41]:
main_df = []
scrape()

tt0114709 Scraped // Films left to scrape = 9625
tt0113497 Scraped // Films left to scrape = 9624
tt0113228 Scraped // Films left to scrape = 9623
tt0114885 Scraped // Films left to scrape = 9622
tt0113041 Scraped // Films left to scrape = 9621
tt0113277 Scraped // Films left to scrape = 9620
tt0114319 Scraped // Films left to scrape = 9619
tt0112302 Scraped // Films left to scrape = 9618
tt0114576 Scraped // Films left to scrape = 9617
tt0113189 Scraped // Films left to scrape = 9616
tt0112346 Scraped // Films left to scrape = 9615
tt0112896 Scraped // Films left to scrape = 9614
tt0112453 Scraped // Films left to scrape = 9613
tt0113987 Scraped // Films left to scrape = 9612
tt0112760 Scraped // Films left to scrape = 9611
tt0112641 Scraped // Films left to scrape = 9610
tt0114388 Scraped // Films left to scrape = 9609
tt0113101 Scraped // Films left to scrape = 9608
tt0112281 Scraped // Films left to scrape = 9607
tt0113845 Scraped // Films left to scrape = 9606
tt0113161 Scraped //

tt0114558 Scraped // Films left to scrape = 9458
tt0058450 Scraped // Films left to scrape = 9457
tt0114663 Scraped // Films left to scrape = 9456
tt0114702 Scraped // Films left to scrape = 9455
tt0114682 Scraped // Films left to scrape = 9454
tt0114781 Scraped // Films left to scrape = 9453
tt0114798 Scraped // Films left to scrape = 9452
tt0114805 Scraped // Films left to scrape = 9451
tt0114887 Scraped // Films left to scrape = 9450
tt0114898 Scraped // Films left to scrape = 9449
tt0114928 Scraped // Films left to scrape = 9448
tt0114938 Scraped // Films left to scrape = 9447
tt0109340 Scraped // Films left to scrape = 9446
tt0112602 Scraped // Films left to scrape = 9445
tt0111579 Scraped // Films left to scrape = 9444
tt0110882 Scraped // Films left to scrape = 9443
tt0112471 Scraped // Films left to scrape = 9442
tt0112508 Scraped // Films left to scrape = 9441
tt0112438 Scraped // Films left to scrape = 9440
tt0112571 Scraped // Films left to scrape = 9439
tt0112757 Scraped //

tt0111256 Scraped // Films left to scrape = 9290
tt0111438 Scraped // Films left to scrape = 9289
tt0111503 Scraped // Films left to scrape = 9288
tt0111693 Scraped // Films left to scrape = 9287
tt0111742 Scraped // Films left to scrape = 9286
tt0111756 Scraped // Films left to scrape = 9285
tt0112443 Scraped // Films left to scrape = 9284
tt0110455 Scraped // Films left to scrape = 9283
tt0111048 Scraped // Films left to scrape = 9282
tt0110399 Scraped // Films left to scrape = 9281
tt0112570 Scraped // Films left to scrape = 9280
tt0109454 Scraped // Films left to scrape = 9279
tt0059170 Scraped // Films left to scrape = 9278
tt0110186 Scraped // Films left to scrape = 9277
tt0111301 Scraped // Films left to scrape = 9276
tt0110027 Scraped // Films left to scrape = 9275
tt0109785 Scraped // Films left to scrape = 9274
tt0113409 Scraped // Films left to scrape = 9273
tt0109021 Scraped // Films left to scrape = 9272
tt0109035 Scraped // Films left to scrape = 9271
tt0106220 Scraped //

tt0109382 Scraped // Films left to scrape = 9123
tt0112572 Scraped // Films left to scrape = 9122
tt0099785 Scraped // Films left to scrape = 9121
tt0099653 Scraped // Films left to scrape = 9120
tt0103639 Scraped // Films left to scrape = 9119
tt0103064 Scraped // Films left to scrape = 9118
tt0099348 Scraped // Films left to scrape = 9117
tt0096895 Scraped // Films left to scrape = 9116
tt0102926 Scraped // Films left to scrape = 9115
tt0029583 Scraped // Films left to scrape = 9114
tt0101414 Scraped // Films left to scrape = 9113
tt0032910 Scraped // Films left to scrape = 9112
tt0100405 Scraped // Films left to scrape = 9111
tt0065214 Scraped // Films left to scrape = 9110
tt0110395 Scraped // Films left to scrape = 9109
tt0109934 Scraped // Films left to scrape = 9108
tt0117247 Scraped // Films left to scrape = 9107
tt0112625 Scraped // Films left to scrape = 9106
tt0116282 Scraped // Films left to scrape = 9105
tt0116552 Scraped // Films left to scrape = 9104
tt0082509 Scraped //

tt0115725 Scraped // Films left to scrape = 8956
tt0115783 Scraped // Films left to scrape = 8955
tt0113596 Scraped // Films left to scrape = 8954
tt0109001 Scraped // Films left to scrape = 8953
tt0113253 Scraped // Films left to scrape = 8952
tt0117991 Scraped // Films left to scrape = 8951
tt0117093 Scraped // Films left to scrape = 8950
tt0111019 Scraped // Films left to scrape = 8949
tt0035896 Encountered ValueError // Films left to scrape = 8948
tt0032904 Scraped // Films left to scrape = 8947
tt0045152 Scraped // Films left to scrape = 8946
tt0043278 Scraped // Films left to scrape = 8945
tt0050419 Scraped // Films left to scrape = 8944
tt0054698 Scraped // Films left to scrape = 8943
tt0052357 Scraped // Films left to scrape = 8942
tt0047396 Scraped // Films left to scrape = 8941
tt0025316 Scraped // Films left to scrape = 8940
tt0036855 Scraped // Films left to scrape = 8939
tt0025164 Scraped // Films left to scrape = 8938
tt0053125 Scraped // Films left to scrape = 8937
tt005

...

tt0080037 Scraped // Films left to scrape = 823
tt0080439 Scraped // Films left to scrape = 822
tt0419058 Scraped // Films left to scrape = 821
tt3079380 Scraped // Films left to scrape = 820
tt3152624 Scraped // Films left to scrape = 819
tt0085204 Encountered ValueError // Films left to scrape = 818
tt3817848 Scraped // Films left to scrape = 817
tt1243974 Scraped // Films left to scrape = 816
tt3672840 Scraped // Films left to scrape = 815
tt1674771 Scraped // Films left to scrape = 814
tt4393514 Scraped // Films left to scrape = 813
tt3133722 Scraped // Films left to scrape = 812
tt1924273 Scraped // Films left to scrape = 811
tt0045708 Scraped // Films left to scrape = 810
tt2096673 Scraped // Films left to scrape = 809
tt2415458 Scraped // Films left to scrape = 808
tt3043546 Scraped // Films left to scrape = 807
tt0903657 Scraped // Films left to scrape = 806
tt1951266 Scraped // Films left to scrape = 805
tt2120120 Scraped // Films left to scrape = 804
tt3183660 Scraped // Film

tt1172049 Scraped // Films left to scrape = 653
tt0643109 Scraped // Films left to scrape = 652
tt1018765 Scraped // Films left to scrape = 651
tt0462335 Scraped // Films left to scrape = 650
tt3530002 Scraped // Films left to scrape = 649
tt2625810 Scraped // Films left to scrape = 648
tt3067274 Scraped // Films left to scrape = 647
tt2719848 Scraped // Films left to scrape = 646
tt3792960 Scraped // Films left to scrape = 645
tt1895587 Scraped // Films left to scrape = 644
tt1596345 Scraped // Films left to scrape = 643
tt3072482 Scraped // Films left to scrape = 642
tt2503944 Scraped // Films left to scrape = 641
tt4176776 Scraped // Films left to scrape = 640
tt0315152 Scraped // Films left to scrape = 639
tt2965466 Scraped // Films left to scrape = 638
tt3702996 Scraped // Films left to scrape = 637
tt0453671 Scraped // Films left to scrape = 636
tt3642618 Scraped // Films left to scrape = 635
tt2510894 Scraped // Films left to scrape = 634
tt3475596 Scraped // Films left to scrap

tt0380249 Scraped // Films left to scrape = 483
tt4104054 Scraped // Films left to scrape = 482
tt2400291 Scraped // Films left to scrape = 481
tt5326448 Scraped // Films left to scrape = 480
tt1974419 Scraped // Films left to scrape = 479
tt1971571 Scraped // Films left to scrape = 478
tt2057392 Scraped // Films left to scrape = 477
tt4991652 Scraped // Films left to scrape = 476
tt2091935 Scraped // Films left to scrape = 475
tt0134854 Scraped // Films left to scrape = 474
tt4136084 Scraped // Films left to scrape = 473
tt3760922 Scraped // Films left to scrape = 472
tt4899406 Scraped // Films left to scrape = 471
tt1468846 Scraped // Films left to scrape = 470
tt4139124 Scraped // Films left to scrape = 469
tt2507238 Scraped // Films left to scrape = 468
tt5480340 Scraped // Films left to scrape = 467
tt2937696 Scraped // Films left to scrape = 466
tt3544112 Scraped // Films left to scrape = 465
tt3763866 Scraped // Films left to scrape = 464
tt5247022 Scraped // Films left to scrap

tt6023444 Scraped // Films left to scrape = 312
tt1725969 Scraped // Films left to scrape = 311
tt0210044 Scraped // Films left to scrape = 310
tt4034228 Scraped // Films left to scrape = 309
tt3741834 Scraped // Films left to scrape = 308
tt5254868 Scraped // Films left to scrape = 307
tt6016776 Scraped // Films left to scrape = 306
tt1798603 Scraped // Films left to scrape = 305
tt4431208 Scraped // Films left to scrape = 304
tt4964772 Scraped // Films left to scrape = 303
tt1542768 Scraped // Films left to scrape = 302
tt0415481 Scraped // Films left to scrape = 301
tt6156350 Scraped // Films left to scrape = 300
tt5555502 Scraped // Films left to scrape = 299
tt2654430 Scraped // Films left to scrape = 298
tt0756316 Scraped // Films left to scrape = 297
tt3260022 Scraped // Films left to scrape = 296
tt5323662 Scraped // Films left to scrape = 295
tt3521164 Scraped // Films left to scrape = 294
tt1711525 Scraped // Films left to scrape = 293
tt3922818 Scraped // Films left to scrap

tt7044010 Scraped // Films left to scrape = 142
tt4633690 Scraped // Films left to scrape = 141
tt1959563 Scraped // Films left to scrape = 140
tt0069369 Scraped // Films left to scrape = 139
tt5135434 Scraped // Films left to scrape = 138
tt6354108 Scraped // Films left to scrape = 137
tt0273646 Scraped // Films left to scrape = 136
tt7158814 Scraped // Films left to scrape = 135
tt4649466 Scraped // Films left to scrape = 134
tt1856101 Scraped // Films left to scrape = 133
tt3486626 Scraped // Films left to scrape = 132
tt1835980 Scraped // Films left to scrape = 131
tt3469518 Scraped // Films left to scrape = 130
tt5109784 Scraped // Films left to scrape = 129
tt6333060 Scraped // Films left to scrape = 128
tt4126568 Scraped // Films left to scrape = 127
tt2492564 Scraped // Films left to scrape = 126
tt1332537 Scraped // Films left to scrape = 125
tt3532216 Scraped // Films left to scrape = 124
tt5030402 Scraped // Films left to scrape = 123
tt1981128 Scraped // Films left to scrap